<a href="https://colab.research.google.com/github/Aakaey181/RA_Porject_ML1/blob/main/app_withdrag_v1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit streamlit-drawable-canvas
!pip install streamlit pyngrok

In [ ]:
!pip install streamlit streamlit-cropper pyngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pkill -f streamlit

In [ ]:
%%writefile app1.py
import streamlit as st
from PIL import Image
from streamlit_cropper import st_cropper
import numpy as np
import tensorflow as tf
import cv2

# Define available models and their paths
model_options = {
    "Clean MobileNet": "drive/MyDrive/ra/trained_models/clean_mobilenet_binary_classifier.h5",
    "Basic MobileNet": "drive/MyDrive/ra/trained_models/mobilenet_binary_classifier.h5"
}

# Model selection widget
selected_model_name = st.selectbox("Choose a Model", list(model_options.keys()))
model_path = model_options[selected_model_name]
model = tf.keras.models.load_model(model_path)

st.title("Zebra Crossing Detector (Cropper Demo)")

# Initialize session state for cropped image if not already set
if "cropped_img" not in st.session_state:
    st.session_state["cropped_img"] = None

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png", "webp"])
if uploaded_file is not None:
    # Load the uploaded image and display it
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_container_width=True)

    st.write("Use the crop box below to select the region of interest (then click **Crop**):")
    # Disable real-time update so the crop is only finalized when clicking Crop
    cropped_img = st_cropper(
        image,
        realtime_update=False,
        box_color="red",
        aspect_ratio=None
    )

    # Button to confirm the crop; store the cropped image in session state.
    if st.button("Crop"):
        st.session_state["cropped_img"] = cropped_img
        st.success("Image cropped!")

    # If a cropped image is available in session state, display it and show Predict button.
    if st.session_state["cropped_img"] is not None:
        st.write("Cropped image preview:")
        st.image(st.session_state["cropped_img"], use_container_width=True)

        if st.button("Predict on Cropped Region"):
            # Preprocess the cropped image for prediction
            resized = st.session_state["cropped_img"].resize((224, 224))
            arr = np.array(resized).astype("float32") / 255.0
            arr = np.expand_dims(arr, axis=0)

            prob = model.predict(arr)[0][0]
            conf = prob * 100
            st.write(f"Prediction: {conf:.2f}% confidence that a zebra crossing is present.")


Overwriting app1.py


In [ ]:
!ngrok authtoken 2vCjd1FQD3nj46KDGwA02cD8fmE_2JU7HFoujPjiYnTW9xDDR

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!nohup streamlit run app1.py &
from pyngrok import ngrok

# Specify the protocol as http for the tunnel
public_url = ngrok.connect(8501, "http")

print("Public URL:", public_url)

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://5126-34-106-85-94.ngrok-free.app" -> "http://localhost:8501"
